# Object Detection with SSD
### Here we demostrate detection on example images using SSD with PyTorch

In [1]:
import torch.nn as nn
from torch.autograd import Variable
from box_utils import*
import torchvision.transforms as transforms
from torch.utils.serialization import load_lua
import PIL
from PIL import Image
from ssd import build_ssd
import torch.backends.cudnn as cudnn
import cv2

ModuleNotFoundError: No module named 'cv2'

## Build SSD300 in Test Phase
1. Build the architecture, specifyingsize of the input image (300),
    and number of object classes to score (21 for VOC dataset)
2. Next we load pretrained weights on the VOC 2007 dataset  

In [ ]:
net = build_ssd('test', 300, 21)    # initialize SSD

net.load_weights('weights/ssd_300_voc07.pkl')  # load weights by specifying location of either .pkl or.t7 file 

## Load Image 
### Here we just load an example image provided in the data folder

In [ ]:
img = load_lua('data/example.t7')
image = Image.open('./data/example.jpg').convert('RGB') 
# Using the torchvision package, we can create a Compose of multiple 
# built-in transorm ops to apply 
t = transforms.Compose([
        transforms.Scale(300),
        transforms.CenterCrop(300),

    ])
# View the transformed input image 
t(image) 

## SSD Forward Pass
### Now just wrap the image in a Variable so it is recognized by PyTorch autograd

In [ ]:
x = img.float()
x = Variable(x) # wrap tensor in Variable
y = net(x)      # forward pass 

## Parse the Detections
Filter outputs with confidence scores lower than a threshold 
Here we choose 60% 

In [ ]:
from data import VOC_CLASSES as labelmap
import numpy as np

def get_labelname(labelmap, top_label_indices):
    return [labelmap[int(l)] for l in top_label_indices]

detections = y.data.numpy()
# Parse the outputs.
det_label = detections[0,:,1]
det_conf = detections[0,:,2]
det_xmin = detections[0,:,3]
det_ymin = detections[0,:,4]
det_xmax = detections[0,:,5]
det_ymax = detections[0,:,6]

# Get detections with confidence higher than 0.6.
top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]

top_conf = det_conf[top_indices]
top_label_indices = det_label[top_indices]
top_labels = get_labelname(labelmap, top_label_indices)
top_xmin = det_xmin[top_indices]
top_ymin = det_ymin[top_indices]
top_xmax = det_xmax[top_indices]
top_ymax = det_ymax[top_indices]

## View Results 

In [ ]:
import matplotlib.pyplot as plt

colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
plt.imshow(image)  # plot the image for matpltlib
currentAxis = plt.gca()
# scale each detection back up to the image
for i in range(top_conf.shape[0]):
    xmin = int(round(top_xmin[i] * image.size[0]))
    ymin = int(round(top_ymin[i] * image.size[1]))
    xmax = int(round(top_xmax[i] * image.size[0]))
    ymax = int(round(top_ymax[i] * image.size[1]))
    score = top_conf[i]
    label = int(top_label_indices[i])
    label_name = top_labels[i]
    display_txt = '%s: %.2f'%(label_name, score)
    coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
    color = colors[label]
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
    currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})

In [ ]:
plt.show()  # display label, score, and bounding boxes!